In [1]:
import requests
from typing import Set

# --- Global lists to be populated with threat intelligence ---
KNOWN_BAD_DOMAINS: Set[str] = set()
KNOWN_BAD_IPS: Set[str] = set()
SUSPICIOUS_KEYWORDS: Set[str] = {
    'urgent', 'password', 'invoice', 'payment', 'suspended', 
    'verify', 'action required', 'click here'
}

# --- Standard Incident Data Structure (for reference) ---
def create_incident_alert(alert_id: str, source_ip: str, sender_domain: str, sender_email: str, subject: str, body: str) -> dict:
    """Creates a standardized incident alert dictionary for testing."""
    return {
        "alert_id": alert_id,
        "source_ip": source_ip,
        "sender_domain": sender_domain,
        "sender_email": sender_email,
        "subject": subject,
        "body": body,
        "triage_score": 0,
        "triage_summary": []
    }

In [2]:
# --- ACTIONABLE: This function fetches real-time threat intel. ---
def fetch_threat_intel_list(url: str, comment_prefix: str = "#") -> Set[str]:
    """
    Fetches a list of IoCs from a URL, ignoring comments.
    
    Args:
        url (str): The URL of the threat intelligence feed.
        comment_prefix (str): The character prefix for comment lines to ignore.

    Returns:
        Set[str]: A set of unique IoCs (domains, IPs, etc.).
    """
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        lines = response.text.splitlines()
        ioc_list = {
            line.strip() 
            for line in lines 
            if line.strip() and not line.startswith(comment_prefix)
        }
        print(f"Successfully fetched {len(ioc_list)} IoCs from {url}")
        return ioc_list
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return set()

In [9]:
# --- ACTIONABLE: Populate our threat intel lists from public feeds. ---
import os
import requests
import json
import sys
def query_urlhaus(auth_key: str, url: str) -> None:
    """
    Queries the URLhaus API for known bad domains related to a specific URL.
    
    Args:
        auth_key (str): The API key for authentication.
        url (str): The URL to analyze.
    """
    api_url = f"https://urlhaus.abuse.ch/api/v1/url/{url}"
    headers = {
        "Accept": "application/json",
        "API-KEY": auth_key
    }
    
    try:
        response = requests.get(api_url, headers=headers, timeout=15)
        response.raise_for_status()
        
        data = response.json()
        if data.get("success"):
            for entry in data.get("url_list", []):
                KNOWN_BAD_DOMAINS.add(entry['url'])
            print(f"Found {len(data.get('url_list', []))} known bad domains for {url}")
        else:
            print(f"No known bad domains found for {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error querying URLhaus: {e}")
print("Populating RECENT KNOWN_BAD_DOMAINS from abuse.ch...")
url_to_analyze = sender_domain
auth_key = os.getenv("URLHAUS_AUTH_KEY")

if auth_key:
    query_urlhaus(auth_key, url_to_analyze)
else:
    print("API key is not available. Please set the environment variable.")

urlhaus_url = "https://urlhaus.abuse.ch/downloads/hostnames/"
KNOWN_BAD_DOMAINS.update(fetch_threat_intel_list(urlhaus_url))

print("\nPopulating KNOWN_BAD_IPS from a placeholder feed...")
# You would replace this with a real feed URL for malicious IPs, e.g., from Spamhaus or SANS.
dummy_ip_feed_url = "http://localhost/ip_feed.txt" # This will fail, as it's a dummy URL
# KNOWN_BAD_IPS.update(fetch_threat_intel_list(dummy_ip_feed_url))

# Print a sample to verify the data
print(f"\nSample of KNOWN_BAD_DOMAINS ({len(KNOWN_BAD_DOMAINS)} total):")
print(list(KNOWN_BAD_DOMAINS)[:5])

Populating RECENT KNOWN_BAD_DOMAINS from abuse.ch...


NameError: name 'sender_domain' is not defined

In [8]:
# --- ACTIONABLE: This is the core logic of the playbook. ---
def initial_phishing_triage(incident: dict) -> dict:
    """
    Performs initial triage on a phishing-related incident using known IoCs.
    
    Args:
        incident (dict): The incident alert to analyze.
        
    Returns:
        dict: The updated incident dictionary with triage scores and summary.
    """
    triage_score = 0
    triage_summary = []
    
    # 1. Check against Known Bad Domains (MITRE ATT&CK T1566.001)
    if incident['sender_domain'] in KNOWN_BAD_DOMAINS:
        triage_score += 50
        triage_summary.append("MATCH: Sender domain is a known bad domain.")
        
    # 2. Check against Known Bad IPs (MITRE ATT&CK T1071.001)
    if incident['source_ip'] in KNOWN_BAD_IPS:
        triage_score += 30
        triage_summary.append("MATCH: Source IP is a known malicious IP address.")
        
    # 3. Check for suspicious keywords in subject and body
    combined_text = incident['subject'] + " " + incident['body']
    found_keywords = [kw for kw in SUSPICIOUS_KEYWORDS if kw in combined_text.lower()]
    
    if found_keywords:
        triage_score += 10 * len(found_keywords)
        triage_summary.append(f"MATCH: Found suspicious keywords: {', '.join(found_keywords)}.")
        
    # Set the final score and summary
    incident['triage_score'] = triage_score
    incident['triage_summary'] = triage_summary
    
    return incident

In [5]:
# --- SIMULATED: This represents a real alert arriving from a security tool. ---
phishing_alert = {
    "alert_id": "PHISH-1002",
    "source_ip": "1.2.3.4", # Placeholder IP
    "sender_domain": "fakewebsite-update.net", # This will trigger a match with our mock list
    "sender_email": "noreply@fakewebsite-update.net",
    "subject": "ACTION REQUIRED: Your password has expired.",
    "body": "Please click the link below to verify your account."
}

print("New incident alert received:")
print(phishing_alert)

New incident alert received:
{'alert_id': 'PHISH-1002', 'source_ip': '1.2.3.4', 'sender_domain': 'fakewebsite-update.net', 'sender_email': 'noreply@fakewebsite-update.net', 'subject': 'ACTION REQUIRED: Your password has expired.', 'body': 'Please click the link below to verify your account.'}


In [6]:
# --- ACTIONABLE: Run the playbook and print the results. ---
triage_result = initial_phishing_triage(phishing_alert)

# Define severity thresholds
if triage_result['triage_score'] >= 50:
    severity = "High"
    mitre_techniques = "T1566.001 (Spearphishing Link)"
elif triage_result['triage_score'] >= 30:
    severity = "Medium"
    mitre_techniques = "T1071.001 (Application Layer Protocol)"
else:
    severity = "Low"
    mitre_techniques = "N/A"
    
print("\n--- Triage Results ---")
print(f"Incident ID: {triage_result['alert_id']}")
print(f"Calculated Score: {triage_result['triage_score']}")
print(f"Severity Level: {severity}")
print(f"MITRE ATT&CK Mapping: {mitre_techniques}")
print("Triage Summary:")
for entry in triage_result['triage_summary']:
    print(f"- {entry}")

if severity == "High":
    print("\nNext Action: This incident requires immediate containment. Proceed to blocking the domain and notifying the affected user.")


--- Triage Results ---
Incident ID: PHISH-1002
Calculated Score: 30
Severity Level: Medium
MITRE ATT&CK Mapping: T1071.001 (Application Layer Protocol)
Triage Summary:
- MATCH: Found suspicious keywords: action required, verify, password.
